In [26]:
import os

## =====================================================================================
## This is a temporarly fix for the freezing and the cuda issues. You can add this
## utility script instead of kaggle_l5kit until Kaggle resolve these issues.
## 
## You will be able to train and submit your results, but not all the functionality of
## l5kit will work properly.

## More details here:
## https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/discussion/177125

## this script transports l5kit and dependencies
os.system('pip install --target=/kaggle/working pymap3d==2.1.0')
os.system('pip install --target=/kaggle/working protobuf==3.12.2')
os.system('pip install --target=/kaggle/working transforms3d')
os.system('pip install --target=/kaggle/working zarr')
os.system('pip install --target=/kaggle/working ptable')

os.system('pip install --no-dependencies --target=/kaggle/working l5kit')
# os.system('pip install --target=/kaggle/working timm')

0

In [14]:
!pip install timm

In [27]:
# import packages
import os, gc
import zarr
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from typing import Dict
from collections import Counter
from prettytable import PrettyTable
from collections import OrderedDict
import math
import pickle
from IPython.display import FileLink

#level5 toolkit
from l5kit.data import PERCEPTION_LABELS
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.data import ChunkedDataset, LocalDataManager

# level5 toolkit 
from l5kit.configs import load_config_data
from l5kit.geometry import transform_points
from l5kit.rasterization import build_rasterizer
from l5kit.visualization import draw_trajectory, draw_reference_trajectory, TARGET_POINTS_COLOR, PREDICTED_POINTS_COLOR, write_gif
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset, export_zarr_to_csv, write_gt_csv
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from colorama import Fore, Back, Style

# deep learning
import torch
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18, resnet50, resnet34
from torchvision.models.mobilenet import mobilenet_v2
from torch.nn import functional as F
import timm

# check files in directory
print((os.listdir('../input/lyft-motion-prediction-autonomous-vehicles/')))

plt.rc('animation', html='jshtml')

%matplotlib inline

['multi_mode_sample_submission.csv', 'semantic_map', 'aerial_map', 'single_mode_sample_submission.csv', 'meta.json', 'scenes']


# Train Configurations

In [28]:
DEBUG = False

# training cfg
training_cfg = {
    
    'format_version': 4,
    
     ## Model options
    'model_params': {
        'model_architecture': 'mixnet_l',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    ## Input raster parameters
    'raster_params': {
        
        'raster_size': [300, 300], # raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.
        'pixel_size': [0.5, 0.5], # From 0 to 1 per axis, [0.5,0.5] would show the ego centered in the image.
        'ego_center': [0.25, 0.5],
        'map_type': "py_semantic",
        
        # the keys are relative to the dataset environment variable
        'satellite_map_key': "aerial_map/aerial_map.png",
        'semantic_map_key': "semantic_map/semantic_map.pb",
        'dataset_meta_key': "meta.json",

        # e.g. 0.0 include every obstacle, 0.5 show those obstacles with >0.5 probability of being
        # one of the classes we care about (cars, bikes, peds, etc.), >=1.0 filter all other agents.
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False

    },

    ## Data loader options
    'train_data_loader': {
        'key': "scenes/train.zarr",
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },

    ## Train params
    'train_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 100 if DEBUG else 10000
    }
}


In [29]:
# root directory
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
print(training_cfg)

{'format_version': 4, 'model_params': {'model_architecture': 'mixnet_l', 'history_num_frames': 10, 'history_step_size': 1, 'history_delta_time': 0.1, 'future_num_frames': 50, 'future_step_size': 1, 'future_delta_time': 0.1}, 'raster_params': {'raster_size': [300, 300], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5, 'disable_traffic_light_faces': False}, 'train_data_loader': {'key': 'scenes/train.zarr', 'batch_size': 32, 'shuffle': True, 'num_workers': 4}, 'train_params': {'checkpoint_every_n_steps': 5000, 'max_num_steps': 10000}}


In [32]:
# training cfg
train_cfg = training_cfg["train_data_loader"]

# rasterizer
rasterizer = build_rasterizer(training_cfg, dm)

# dataloader
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(training_cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
print(train_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


# Model Definition

## LyftModel - simple Resnet

In [33]:
class LyftModel(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        
        # set pretrained=True while training
        self.backbone = resnet50(pretrained=True) 
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            # num_in_channels = 25
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 2048
        
        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.logit = nn.Linear(4096, out_features=num_targets)
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        
        return x

## Mixnet

In [34]:
class LyftMixnet(nn.Module):
    
    def __init__(self, cfg, classify='mixnet_l'):
        super().__init__()
        
        # set pretrained=True while training
        self.backbone = timm.create_model(classify, pretrained=True) 
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        
        self.backbone.conv_stem = nn.Conv2d(
            # num_in_channels = 25
            num_in_channels,
            self.backbone.conv_stem.out_channels,
            kernel_size=self.backbone.conv_stem.kernel_size,
            stride=self.backbone.conv_stem.stride,
            padding=self.backbone.conv_stem.padding,
            bias=False,
        )
        
        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]
        self.logit = nn.Linear(self.backbone.classifier.out_features, out_features=num_targets)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.logit(x)
        return x

## MobileNet v2

In [35]:
class LyftMobile(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        
        # set pretrained=True while training
        self.backbone = mobilenet_v2(pretrained=True) 
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.features[0][0] = nn.Conv2d(
            # num_in_channels = 25
            num_in_channels,
            self.backbone.features[0][0].out_channels,
            kernel_size=self.backbone.features[0][0].kernel_size,
            stride=self.backbone.features[0][0].stride,
            padding=self.backbone.features[0][0].padding,
            bias=False,
        )
                
        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # Fully connected layer.
        self.backbone.classifier[1] = nn.Linear(
            in_features=self.backbone.classifier[1].in_features,
            out_features=num_targets
        )

        
    def forward(self, x):
        x = self.backbone(x)
        return x

## Multi-Modal

In [36]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py

def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    if not torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))):
        print(confidences)
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## LSTM layer

# Hyperparameters

In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# choose parameter conf
conf = 2

if conf == 1:
    # resnet stuff
    model = LyftModel(training_cfg).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=2e-6)
    criterion = nn.SmoothL1Loss()
    # criterion = nn.MSELoss(reduction="none")

if conf == 2:
    # mixnet_large

    model = LyftMixnet(training_cfg, 'mixnet_l').to(device)
    
    WEIGHT_FILE = '../input/resnet-34-pth/model_state_mixnetl_25000.pth'
    model_state = torch.load(WEIGHT_FILE, map_location=device)
    model.load_state_dict(model_state)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=2e-6)
    criterion = pytorch_neg_multi_log_likelihood_single
    
if conf == 3:
    # mixnet_medium
    model = LyftMixnet(training_cfg, 'mixnet_m').to(device)
    optimizer = optim.Adam(model.parameters(), lr=4e-4, weight_decay=2e-6)
    criterion = nn.SmoothL1Loss()

if conf == 4:
    model = LyftMobile(training_cfg).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=2e-6)
    criterion = nn.SmoothL1Loss()


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.90 GiB total capacity; 15.00 GiB already allocated; 3.75 MiB free; 15.13 GiB reserved in total by PyTorch)

# Validate Configuration

In [24]:
# validate cfg
validate_cfg = {
    
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [300, 300],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },
    
    'validate_data_loader': {
    'key': 'scenes/validate.zarr',
    'batch_size': 6,
    'shuffle': False,
    'num_workers': 4
    }

}

In [31]:
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(training_cfg["train_params"]["max_num_steps"]))

losses_train = []
likelihood_valid = []

for iter_step in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    model.train()
    torch.set_grad_enabled(True)
    
    # forward pass
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    
    outputs = model(inputs).reshape(targets.shape).to(device)
    # use Negative Multi Log Likelihood
    loss = criterion(targets, outputs, target_availabilities.squeeze(-1))

#     loss = criterion(outputs, targets)
#     # not all the output steps are valid, but we can filter them out from the loss using availabilities
#     loss = loss * target_availabilities
#     loss = loss.mean()

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
        
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")
    

  0%|          | 0/10000 [00:06<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 28.00 MiB (GPU 0; 15.90 GiB total capacity; 14.97 GiB already allocated; 9.75 MiB free; 15.13 GiB reserved in total by PyTorch)

## Loss Plot & NLL Plot

In [ ]:
# Training Loss plot
l = [i for i in range(1, len(losses_train)+1)]
# new_ticks=np.linspace(0,training_cfg["train_params"]["max_num_steps"],6)
plt.plot(l, losses_train,label="Training Loss")
# plt.xticks(new_ticks)
plt.title("Loss Performance Versus Scenes")
plt.rcParams['figure.figsize'] = 10, 10
plt.xlabel("Scenes")
plt.ylabel("Loss")
plt.savefig("loss_mixnet_l_45000.png")
plt.show()

In [ ]:
# Validation Negative Log Likelihood Plot
# l = [i for i in range(len(likelihood_valid))]
# plt.plot(l, likelihood_valid,label="NLL")
# plt.xticks(l, [str(i * 1000) for i in l])
# plt.title("Validation Negative Log Likelihood Versus Scenes")
# plt.rcParams['figure.figsize'] = 10, 10
# plt.xlabel("Scenes")
# plt.ylabel("NLL")
# plt.show()

## Save Model

In [ ]:
# save full trained model
model_name = "model_state_mixnetl_45000.pth"

torch.save(model.state_dict(), model_name)

In [ ]:
os.chdir(r'/kaggle/working')
FileLink(model_name)

In [ ]:
FileLink("loss_mixnet_l_45000.png")
# gc.collect()

In [ ]:
# # validation configuration
# valid_cfg = validate_cfg["validate_data_loader"]

# # Rasterizer
# rasterizer = build_rasterizer(validate_cfg, dm)

# # Validation dataset/dataloader
# valid_zarr = ChunkedDataset(dm.require(valid_cfg["key"])).open()
# # valid_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
# valid_dataset = AgentDataset(validate_cfg, valid_zarr, rasterizer)
# whole_size = valid_dataset.__len__()
# valid_dataset_use, valid_dataset_valid, _ = torch.utils.data.random_split(valid_dataset, [70000, 2000, whole_size-72000], generator=torch.Generator().manual_seed(42))

# valid_dataloader = DataLoader(valid_dataset_use,
#                              shuffle=valid_cfg["shuffle"],
#                              batch_size=valid_cfg["batch_size"],
#                              num_workers=valid_cfg["num_workers"])

# valid_dataloader_valid = DataLoader(valid_dataset_valid,
#                              shuffle=valid_cfg["shuffle"],
#                              batch_size=valid_cfg["batch_size"],
#                              num_workers=valid_cfg["num_workers"])

# print(valid_dataloader.dataset.__len__())


# Load Model

In [ ]:
# # Saved state dict from the training notebook
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# WEIGHT_FILE = '../input/resnet-34-pth/model_state_mixnetl_15000.pth'
# model_state = torch.load(WEIGHT_FILE, map_location=device)
# model.load_state_dict(model_state)

# Final Evaluation

In [ ]:
# # Final - Evaluate Validation Dataset 
# model.eval()
# torch.set_grad_enabled(False)

# # store information for evaluation
# future_coords_offsets_pd = []
# timestamps = []
# # coordinates ground truth
# valid_coords_gts = []
# # target avalabilities
# target_avail_pd = []
# agent_ids = []
# progress_bar = tqdm(valid_dataloader)
# for data in progress_bar:
    
#     inputs = data["image"].to(device)
#     target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
#     targets = data["target_positions"].to(device)
    
#     outputs = model(inputs).reshape(targets.shape)
    
#     future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
#     timestamps.append(data["timestamp"].numpy().copy())
#     agent_ids.append(data["track_id"].numpy().copy())
#     valid_coords_gts.append(data["target_positions"].numpy().copy())
#     target_avail_pd.append(target_availabilities.cpu().numpy().copy())

In [ ]:
# timestamps_concat = np.concatenate(timestamps)
# track_ids_concat = np.concatenate(agent_ids)
# coords_concat = np.concatenate(future_coords_offsets_pd)
# gt_valid_final = np.concatenate(valid_coords_gts)
# target_avail_concat = np.concatenate(target_avail_pd)

# # gt_valid_2D = np.reshape(gt_valid_final, (70000, 100))

# # log_like = neg_multi_log_likelihood (
# #     ground_truth=gt_valid_2D,
# #     pred=coords_concat,
# #     confidences=np.array([1,0,0]),
# #     avails=target_avail_concat
# # )

# # print("Negative multi Likelihood is:", log_like)

## Ground Truth and Prediction CSV

In [ ]:
# # generate ground truth csv
# write_gt_csv(
#     csv_path="valid_gt.csv", 
#     timestamps=timestamps_concat, 
#     track_ids=track_ids_concat, 
#     coords=gt_valid_final, 
#     avails=target_avail_concat.squeeze(-1)
# )

# # submission.csv
# write_pred_csv('submission_mixnetl_15000.csv',
#                timestamps=timestamps_concat,
#                track_ids=track_ids_concat,
#                coords=coords_concat,
#               )

## Load CSV & Compute metrics

In [ ]:
# # Negative Log Likelihood Metrics
# eval_gt_path = "../input/resnet-34-pth/valid_gt.csv"
# pred_path = "submission_mixnetl_15000.csv"

# metrics = compute_metrics_csv(eval_gt_path, pred_path, [neg_multi_log_likelihood, time_displace])
# for metric_name, metric_mean in metrics.items():
#     print(metric_name, metric_mean)
    
# # Save Metric
# np.save('metric_mixnetl_15000.npy',metrics)

## Plot Prediction Tractories

In [ ]:
# model.eval()
# torch.set_grad_enabled(False)

# # Uncomment to choose satelliter or semantic rasterizer
# validate_cfg["raster_params"]["map_type"] = "py_satellite"
# # validate_cfg["raster_params"]["map_type"] = "py_semantic"


# rast = build_rasterizer(validate_cfg, dm)

# eval_ego_dataset = EgoDataset(validate_cfg, valid_dataset.dataset, rast)
# num_frames = 2 # randomly pick _ frames
# random_frames = np.random.randint(0,len(eval_ego_dataset)-1, (num_frames,))

# for frame_number in random_frames:  
#     agent_indices = valid_dataset.get_frame_indices(frame_number) 
#     if not len(agent_indices):
#         continue

#     # get AV point-of-view frame
#     data_ego = eval_ego_dataset[frame_number]
#     im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
#     center = np.asarray(validate_cfg["raster_params"]["ego_center"]) * validate_cfg["raster_params"]["raster_size"]
    
#     predicted_positions = []
#     target_positions = []

#     for v_index in agent_indices:
#         data_agent = valid_dataset[v_index]

#         out_net = model(torch.from_numpy(data_agent["image"]).unsqueeze(0).to(device))
#         out_pos = out_net[0].reshape(-1, 2).detach().cpu().numpy()
#         # store absolute world coordinates
#         predicted_positions.append(transform_points(out_pos, data_agent["world_from_agent"]))
#         # retrieve target positions from the GT and store as absolute coordinates
#         track_id, timestamp = data_agent["track_id"], data_agent["timestamp"]
#         target_positions.append(transform_points(data_agent["target_positions"], data_agent["world_from_agent"]) )

#     # convert coordinates to AV point-of-view so we can draw them
#     predicted_positions = transform_points(np.concatenate(predicted_positions), data_ego["raster_from_world"])
#     target_positions = transform_points(np.concatenate(target_positions), data_ego["raster_from_world"])
    
#     # make sure ground truth and prediction have the same data size
#     assert len(target_positions) == len(predicted_positions)
    
#     # draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
#     draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
    
#     plt.rcParams['figure.figsize'] = 6, 6
#     plt.imshow(im_ego[::-1])
#     plt.show()
    
#     draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    
#     plt.rcParams['figure.figsize'] = 6, 6
#     plt.imshow(im_ego[::-1])
#     plt.show()